# Creating an interannual monthly zooplankton C that I can save and be able to more easily use in my other figures

In [1]:
import warnings
warnings.simplefilter("ignore") # Silence warnings
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import numpy as np
from matplotlib.pyplot import figure

import numpy.ma as ma
from netCDF4 import Dataset as NetCDFFile
import cartopy
import cartopy.crs as ccrs
import pylab 
from copy import deepcopy
import pandas as pd
import matplotlib
import matplotlib.colors as colors
from cartopy.util import add_cyclic_point
plt.rcParams['mathtext.default']='regular'
from collections import OrderedDict
import cmocean
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib
# Scientific libraries
from numpy import arange,array,ones
from scipy import stats
import os

%matplotlib inline
from glob import glob
import dask
#import esmlab
import pop_tools 
from statistics import mean

In [2]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        if clip is None:
            clip = self.clip

        result, is_scalar = self.process_value(value)

        self.autoscale_None(result)
        vmin, vmax, midpoint = self.vmin, self.vmax, self.midpoint

        if not (vmin < midpoint < vmax):
            raise ValueError("midpoint must be between maxvalue and minvalue.")
        elif vmin == vmax:
            result.fill(0) # Or should it be all masked? Or 0.5?
        elif vmin > vmax:
            raise ValueError("maxvalue must be bigger than minvalue")
        else:
            vmin = float(vmin)
            vmax = float(vmax)
            if clip:
                mask = np.ma.getmask(result)
                result = np.ma.array(np.clip(result.filled(vmax), vmin, vmax),
                                  mask=mask)

            # ma division is very slow; we can take a shortcut
            resdat = result.data

            #First scale to -1 to 1 range, than to from 0 to 1.
            resdat -= midpoint
            resdat[resdat>0] /= abs(vmax - midpoint)
            resdat[resdat<0] /= abs(vmin - midpoint)

            resdat /= 2.
            resdat += 0.5
            result = np.ma.array(resdat, mask=result.mask, copy=False)

        if is_scalar:
            result = result[0]
        return result

In [3]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [4]:
def adjust_pop_grid(tlon,tlat,field):
    nj = tlon.shape[0]
    ni = tlon.shape[1]
    xL = int(ni/2 - 1)
    xR = int(xL + ni)

    tlon = np.where(np.greater_equal(tlon,np.min(tlon[:,0])),tlon-360.,tlon)
    lon  = np.concatenate((tlon,tlon+360.),1)
    lon = lon[:,xL:xR]

    if ni == 320:
        lon[367:-3,0] = lon[367:-3,0]+360.
    lon = lon - 360.
    lon = np.hstack((lon,lon[:,0:1]+360.))
    if ni == 320:
        lon[367:,-1] = lon[367:,-1] - 360.

    #-- trick cartopy into doing the right thing:
    #   it gets confused when the cyclic coords are identical
    lon[:,0] = lon[:,0]-1e-8

    #-- periodicity
    lat  = np.concatenate((tlat,tlat),1)
    lat = lat[:,xL:xR]
    lat = np.hstack((lat,lat[:,0:1]))

    field = np.ma.concatenate((field,field),1)
    field = field[:,xL:xR]
    field = np.ma.hstack((field,field[:,0:1]))
    return lon,lat,field

## Seasonality and interannual variability in SPECTRA

In [5]:
case = 'g.e21.G1850ECOIAF.t62_g17.marbl0_33.GNG595'
path = f'/glade/campaign/cesm/development/bgcwg/projects/marbl-spectra/{case}/ocn/hist'

In [6]:
variables = [f'{var}C' for var in ['zoo1','zoo2','zoo3', 'zoo4', 'zoo5', 'zoo6']]
coords = {'x':'TLONG','y':'TLAT'}
keep_vars = variables + list(coords.values())+['dz','KMT','time']

In [7]:
%%time
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
ds_01_avg = xr.Dataset()

for year in np.arange(63,125,1):
    yr4="0{:02d}".format(year).zfill(4)
    print(year)
    for month in months:
    
        ds_01 = xr.Dataset()

        file = sorted(glob(f'{path}/{case}.pop.h.{yr4}-{month}.nc'))
        
        dsv_01=xr.open_mfdataset(file, decode_times=True,drop_variables=["transport_components", "transport_regions"], 
                            parallel=True, compat="override", combine='nested', concat_dim="time",data_vars="minimal",coords='minimal' )
    
        for vv in variables: 
            ds_01 = xr.merge((ds_01, dsv_01[vv]))

        ds_01 = ds_01.drop([v for v in ds_01.variables if v not in keep_vars]).squeeze()
        ds_01_avg = xr.concat([ds_01_avg, ds_01],dim='year')
    

63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
CPU times: user 9min 54s, sys: 6min 51s, total: 16min 45s
Wall time: 19min 40s


# Zooplankton 1

In [8]:
zoo1C_bmss = np.array(ds_01_avg.zoo1C)

In [9]:
%%time
zoo1C_ds_inter = np.empty([62,12,15,384,320])
zoo1C_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    zoo1C_ds_inter[i,:,:,:,:] = zoo1C_bmss[x[i]:x[i]+12,:,:,:]

CPU times: user 1.89 s, sys: 1.83 s, total: 3.71 s
Wall time: 3.78 s


In [10]:
zoo1C_bm_150m = np.nansum(zoo1C_ds_inter, axis=2) * 10/150 #integrate over depth #mmol m-3
zoo1C_bm_150m = zoo1C_bm_150m * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

In [11]:
month = np.arange(0,12,1)
time = np.arange(0,62,1)
zoo1C_bmss_150m = xr.DataArray(zoo1C_bm_150m, coords={'time':time,'month':month, 'TLAT':ds_01_avg.TLAT, 'TLONG':ds_01_avg.TLONG},
                                    dims=['time','month', 'nlat', 'nlon'],
                                    attrs=dict(description="zoo1 integrated over depth (150m)",units="ugC m^{-3}"))
zoo1C_bmss_150m = zoo1C_bmss_150m.to_dataset(name='zoo1C_150m')

In [12]:
zoo1C_bmss_150m.to_netcdf('/glade/derecho/scratch/gabyn/SPECTRA_hindcast/SPECTRA_1990_2009/GNG595_monthly_yearly_1948_2009/zoo1C_62yr_monthly_150m.nc');

# Zooplankton 2

In [13]:
zoo2C_bmss = np.array(ds_01_avg.zoo2C)

In [14]:
%%time
zoo2C_ds_inter = np.empty([62,12,15,384,320])
zoo2C_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    zoo2C_ds_inter[i,:,:,:,:] = zoo2C_bmss[x[i]:x[i]+12,:,:,:]

CPU times: user 1.87 s, sys: 1.85 s, total: 3.72 s
Wall time: 3.81 s


In [15]:
zoo2C_bm_150m = np.nansum(zoo2C_ds_inter, axis=2) * 10/150 #integrate over depth #mmol m-3
zoo2C_bm_150m = zoo2C_bm_150m * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

In [16]:
month = np.arange(0,12,1)
time = np.arange(0,62,1)
zoo2C_bmss_150m = xr.DataArray(zoo2C_bm_150m, coords={'time':time,'month':month, 'TLAT':ds_01_avg.TLAT, 'TLONG':ds_01_avg.TLONG},
                                    dims=['time','month', 'nlat', 'nlon'],
                                    attrs=dict(description="zoo2 integrated over depth (150m)",units="ugC m^{-3}"))
zoo2C_bmss_150m = zoo2C_bmss_150m.to_dataset(name='zoo2C_150m')

In [17]:
zoo2C_bmss_150m.to_netcdf('/glade/derecho/scratch/gabyn/SPECTRA_hindcast/SPECTRA_1990_2009/GNG595_monthly_yearly_1948_2009/zoo2C_62yr_monthly_150m.nc');

# Zooplankton 3

In [18]:
zoo3C_bmss = np.array(ds_01_avg.zoo3C)

In [19]:
%%time
zoo3C_ds_inter = np.empty([62,12,15,384,320])
zoo3C_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    zoo3C_ds_inter[i,:,:,:,:] = zoo3C_bmss[x[i]:x[i]+12,:,:,:]

CPU times: user 1.85 s, sys: 1.87 s, total: 3.72 s
Wall time: 3.82 s


In [20]:
zoo3C_bm_150m = np.nansum(zoo3C_ds_inter, axis=2) * 10/150 #integrate over depth #mmol m-3
zoo3C_bm_150m = zoo3C_bm_150m * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

In [21]:
month = np.arange(0,12,1)
time = np.arange(0,62,1)
zoo3C_bmss_150m = xr.DataArray(zoo3C_bm_150m, coords={'time':time,'month':month, 'TLAT':ds_01_avg.TLAT, 'TLONG':ds_01_avg.TLONG},
                                    dims=['time','month', 'nlat', 'nlon'],
                                    attrs=dict(description="zoo3 integrated over depth (150m)",units="ugC m^{-3}"))
zoo3C_bmss_150m = zoo3C_bmss_150m.to_dataset(name='zoo3C_150m')

In [22]:
zoo3C_bmss_150m.to_netcdf('/glade/derecho/scratch/gabyn/SPECTRA_hindcast/SPECTRA_1990_2009/GNG595_monthly_yearly_1948_2009/zoo3C_62yr_monthly_150m.nc');

# Zooplankton 4

In [23]:
zoo4C_bmss = np.array(ds_01_avg.zoo4C)

In [24]:
%%time
zoo4C_ds_inter = np.empty([62,12,15,384,320])
zoo4C_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    zoo4C_ds_inter[i,:,:,:,:] = zoo4C_bmss[x[i]:x[i]+12,:,:,:]

CPU times: user 2.14 s, sys: 2.3 s, total: 4.44 s
Wall time: 4.55 s


In [25]:
zoo4C_bm_150m = np.nansum(zoo4C_ds_inter, axis=2) * 10/150 #integrate over depth #mmol m-3
zoo4C_bm_150m = zoo4C_bm_150m * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

In [26]:
month = np.arange(0,12,1)
time = np.arange(0,62,1)
zoo4C_bmss_150m = xr.DataArray(zoo4C_bm_150m, coords={'time':time,'month':month, 'TLAT':ds_01_avg.TLAT, 'TLONG':ds_01_avg.TLONG},
                                    dims=['time','month', 'nlat', 'nlon'],
                                    attrs=dict(description="zoo4 integrated over depth (150m)",units="ugC m^{-3}"))
zoo4C_bmss_150m = zoo4C_bmss_150m.to_dataset(name='zoo4C_150m')

In [28]:
zoo4C_bmss_150m.to_netcdf('/glade/derecho/scratch/gabyn/SPECTRA_hindcast/SPECTRA_1990_2009/GNG595_monthly_yearly_1948_2009/zoo4C_62yr_monthly_150m.nc');

# Zooplankton 5

In [29]:
zoo5C_bmss = np.array(ds_01_avg.zoo5C)

In [30]:
%%time
zoo5C_ds_inter = np.empty([62,12,15,384,320])
zoo5C_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    zoo5C_ds_inter[i,:,:,:,:] = zoo5C_bmss[x[i]:x[i]+12,:,:,:]

CPU times: user 1.99 s, sys: 1.75 s, total: 3.74 s
Wall time: 3.83 s


In [31]:
zoo5C_bm_150m = np.nansum(zoo5C_ds_inter, axis=2) * 10/150 #integrate over depth #mmol m-3
zoo5C_bm_150m = zoo5C_bm_150m * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

In [32]:
month = np.arange(0,12,1)
time = np.arange(0,62,1)
zoo5C_bmss_150m = xr.DataArray(zoo5C_bm_150m, coords={'time':time,'month':month, 'TLAT':ds_01_avg.TLAT, 'TLONG':ds_01_avg.TLONG},
                                    dims=['time','month', 'nlat', 'nlon'],
                                    attrs=dict(description="zoo5 integrated over depth (150m)",units="ugC m^{-3}"))
zoo5C_bmss_150m = zoo5C_bmss_150m.to_dataset(name='zoo5C_150m')

In [33]:
zoo5C_bmss_150m.to_netcdf('/glade/derecho/scratch/gabyn/SPECTRA_hindcast/SPECTRA_1990_2009/GNG595_monthly_yearly_1948_2009/zoo5C_62yr_monthly_150m.nc');

# Zooplankton 6

In [34]:
zoo6C_bmss = np.array(ds_01_avg.zoo6C)

In [35]:
%%time
zoo6C_ds_inter = np.empty([62,12,15,384,320])
zoo6C_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    zoo6C_ds_inter[i,:,:,:,:] = zoo6C_bmss[x[i]:x[i]+12,:,:,:]

CPU times: user 1.93 s, sys: 3.24 s, total: 5.17 s
Wall time: 5.4 s


In [36]:
zoo6C_bm_150m = np.nansum(zoo6C_ds_inter, axis=2) * 10/150 #integrate over depth #mmol m-3
zoo6C_bm_150m = zoo6C_bm_150m * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

In [37]:
month = np.arange(0,12,1)
time = np.arange(0,62,1)
zoo6C_bmss_150m = xr.DataArray(zoo6C_bm_150m, coords={'time':time,'month':month, 'TLAT':ds_01_avg.TLAT, 'TLONG':ds_01_avg.TLONG},
                                    dims=['time','month', 'nlat', 'nlon'],
                                    attrs=dict(description="zoo6 integrated over depth (150m)",units="ugC m^{-3}"))
zoo6C_bmss_150m = zoo6C_bmss_150m.to_dataset(name='zoo6C_150m')

In [38]:
zoo6C_bmss_150m.to_netcdf('/glade/derecho/scratch/gabyn/SPECTRA_hindcast/SPECTRA_1990_2009/GNG595_monthly_yearly_1948_2009/zoo6C_62yr_monthly_150m.nc');

# Regrid to 180 by 360 grid

## 3. Regrid from that nc file (you can do this in bash)
`zoo1C`  

cd /glade/u/home/gabyn/scratch/SPECTRA/nc_files
ncatted -a coordinates,zoo1C_150m,c,c,"TLONG TLAT" zoo1C_62yr_monthly_150m.nc
cdo remapbil,r360x180 -selname,zoo1C_150m zoo1C_62yr_monthly_150m.nc zoo1C_62yr_monthly_150m_regrid.nc
cdo sellonlatbox,-180,180,-89.5,89.5 zoo1C_62yr_monthly_150m_regrid.nc regrid_zoo1C_62yr_monthly_150m.nc

`zoo2C`

ncatted -a coordinates,zoo2C_150m,c,c,"TLONG TLAT" zoo2C_62yr_monthly_150m.nc
cdo remapbil,r360x180 -selname,zoo2C_150m zoo2C_62yr_monthly_150m.nc zoo2C_62yr_monthly_150m_regrid.nc
cdo sellonlatbox,-180,180,-89.5,89.5 zoo2C_62yr_monthly_150m_regrid.nc regrid_zoo2C_62yr_monthly_150m.nc


`zoo3C`

ncatted -a coordinates,zoo3C_150m,c,c,"TLONG TLAT" zoo3C_62yr_monthly_150m.nc
cdo remapbil,r360x180 -selname,zoo3C_150m zoo3C_62yr_monthly_150m.nc zoo3C_62yr_monthly_150m_regrid.nc
cdo sellonlatbox,-180,180,-89.5,89.5 zoo3C_62yr_monthly_150m_regrid.nc regrid_zoo3C_62yr_monthly_150m.nc

`zoo4C`

ncatted -a coordinates,zoo4C_150m,c,c,"TLONG TLAT" zoo4C_62yr_monthly_150m.nc
cdo remapbil,r360x180 -selname,zoo4C_150m zoo4C_62yr_monthly_150m.nc zoo4C_62yr_monthly_150m_regrid.nc
cdo sellonlatbox,-180,180,-89.5,89.5 zoo4C_62yr_monthly_150m_regrid.nc regrid_zoo4C_62yr_monthly_150m.nc

`zoo5C`

ncatted -a coordinates,zoo5C_150m,c,c,"TLONG TLAT" zoo5C_62yr_monthly_150m.nc
cdo remapbil,r360x180 -selname,zoo5C_150m zoo5C_62yr_monthly_150m.nc zoo5C_62yr_monthly_150m_regrid.nc
cdo sellonlatbox,-180,180,-89.5,89.5 zoo5C_62yr_monthly_150m_regrid.nc regrid_zoo5C_62yr_monthly_150m.nc

`zoo6C`

ncatted -a coordinates,zoo6C_150m,c,c,"TLONG TLAT" zoo6C_62yr_monthly_150m.nc
cdo remapbil,r360x180 -selname,zoo6C_150m zoo6C_62yr_monthly_150m.nc zoo6C_62yr_monthly_150m_regrid.nc
cdo sellonlatbox,-180,180,-89.5,89.5 zoo6C_62yr_monthly_150m_regrid.nc regrid_zoo6C_62yr_monthly_150m.nc